<a href="https://colab.research.google.com/github/fmhirwa/ML-Pipeline_Task1/blob/main/APIs_and_Web_scraping_Peer_Learning_Activity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**PLD 1** APIs and Web scraping Peer Learning Activity

Task 0

In [42]:
#Florent
#!/usr/bin/env python3
"""Code to return the list of ships"""

import requests

def availableShips(passengerCount):
    """ List of ships

    Arguments:
        passengerCount (int): number of passengers
    """
    res = requests.get('https://swapi-api.alx-tools.com/api/starships')

    output = []
    while res.status_code == 200:
        res = res.json()
        for ship in res['results']:
            passengers = ship['passengers'].replace(',', '')
            try:
                if int(passengers) >= passengerCount:
                    output.append(ship['name'])
            except ValueError:
                pass
        if res['next']:
            res = requests.get(res['next'])
        else:
            break
    return output

# Testing; For colab only
# Get ships with at least 10 passengers
ships = availableShips(10)
print(ships)

['CR90 corvette', 'Sentinel-class landing craft', 'Death Star', 'Executor', 'Rebel transport', 'Imperial shuttle', 'EF76 Nebulon-B escort frigate', 'Calamari Cruiser', 'Republic Cruiser', 'Droid control ship', 'J-type diplomatic barge', 'AA-9 Coruscant freighter', 'Republic Assault ship', 'Solar Sailer', 'Trade Federation cruiser', 'Theta-class T-2c shuttle', 'Republic attack cruiser']


Scrapping

In [ ]:
!pip install requests beautifulsoup4 pandas


In [ ]:
# Mohamed Yasin
# import Libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL to scrape data from
url = 'https://www.scrapethissite.com/pages/forms/'

# Send a GET request to fetch the page content
response = requests.get(url)

# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(response.content, 'html.parser')

# Find the table containing data
table = soup.find('table')

# Extract headers
headers = [header.text.strip() for header in table.find_all('th')]

# Extract rows
rows = []
for row in table.find_all('tr')[1:]:
    cols = row.find_all('td')
    data = [col.text.strip() for col in cols]
    rows.append(data)

# Create a DataFrame using the headers and rows
df = pd.DataFrame(rows, columns=headers)

# Save the DataFrame to a CSV file
df.to_csv('scraped_data.csv', index=False)

# Display the DataFrame in the output
print("Scraped Data:")
print(df)


Scraped Data:
                Team Name  Year Wins Losses OT Losses  Win % Goals For (GF)  \
0           Boston Bruins  1990   44     24             0.55            299   
1          Buffalo Sabres  1990   31     30            0.388            292   
2          Calgary Flames  1990   46     26            0.575            344   
3      Chicago Blackhawks  1990   49     23            0.613            284   
4       Detroit Red Wings  1990   34     38            0.425            273   
5         Edmonton Oilers  1990   37     37            0.463            272   
6        Hartford Whalers  1990   31     38            0.388            238   
7       Los Angeles Kings  1990   46     24            0.575            340   
8   Minnesota North Stars  1990   27     39            0.338            256   
9      Montreal Canadiens  1990   39     30            0.487            273   
10      New Jersey Devils  1990   32     33              0.4            272   
11     New York Islanders  1990   25  

Amazon Scrapping

In [ ]:
# Florent: Amazon scrapping; Amazon blocks after repeated use;
# Change headers and add other headers to play it

In [49]:
import requests
from bs4 import BeautifulSoup
import os

# Create a directory to save images
os.makedirs('product_images', exist_ok=True)
os.makedirs('product_images/laptops', exist_ok=True)

# Function to scrape products
def scrape_products(search_query):
    # Replace spaces with '+' for the search query
    search_query = search_query.replace(' ', '+')
    url = f'https://www.amazon.com/s?k={search_query}'

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:92.0) Gecko/20100101 Firefox/92.0',
        'Accept-Language': 'en-US, en;q=0.9',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8',
        'Referer': 'https://www.amazon.com/',
    }

    # Send a GET request to fetch the page content
    response = requests.get(url, headers=headers)

    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    products = []
    count = 0

    # Loop through search results and extract product data
    for product in soup.find_all('div', {'data-component-type': 's-search-result'}):
        # Extract product name
        name = product.h2.text.strip()

        # Extract image URL
        image_tag = product.find('img', {'class': 's-image'})
        image_url = image_tag['src'] if image_tag else None

        if name and image_url and count < 5:
            # Save image locally
            image_response = requests.get(image_url)
            image_path = f'product_images/laptops/{name[:30].replace(" ", "_")}.jpg'
            with open(image_path, 'wb') as f:
                f.write(image_response.content)

            products.append((name, image_path))
            count += 1

    return products

# Searching for "cookies"
products = scrape_products("laptops")

# Display product names and image paths
print("Scraped Products:")
for product_name, image_path in products:
    print(f'Product: {product_name}, Image saved at: {image_path}')


Scraped Products:
Product: Acer Aspire 3 A315-24P-R7VH Slim Laptop | 15.6" Full HD IPS Display | AMD Ryzen 3 7320U Quad-Core Processor | AMD Radeon Graphics | 8GB LPDDR5 | 128GB NVMe SSD | Wi-Fi 6 | Windows 11 Home in S Mode, Image saved at: product_images/laptops/Acer_Aspire_3_A315-24P-R7VH_Sl.jpg
Product: HP Newest 255 G10 Laptop for Home or Work, 16GB RAM, 1TB SSD, 15.6" Full HD, Ryzen 3 7330U (Beat Intel i5-1135G7), Ethernet Port, HDMI, USB-C, Windows 11 Pro, Business and Fun Ready (2024), Image saved at: product_images/laptops/HP_Newest_255_G10_Laptop_for_H.jpg
Product: Acer Aspire Go 15 Slim Laptop | 15.6" Full HD IPS 1080P Display | Intel Core i3-N305| Intel UHD Graphics | 8GB LPDDR5 | 128GB HD | Wi-Fi 6 | AI PC | Windows 11 Home in S Mode | AG15-31P-3947, Image saved at: product_images/laptops/Acer_Aspire_Go_15_Slim_Laptop_.jpg
Product: HP Newest 14" Ultral Light Laptop for Students and Business, Intel Quad-Core N4120, 8GB RAM, 192GB Storage(64GB eMMC+128GB Micro SD), 1 Year Of

In [ ]:
os.makedirs('product_images/halloween', exist_ok=True) # additional categories to go to separate directories; example.

In [46]:
# Mohamed Yasin

# import Libraries
import requests
from bs4 import BeautifulSoup
import os

# Create a directory to save images
os.makedirs('product_images/halloween', exist_ok=True)

# Function to scrape products
def scrape_products(search_query):
    # Replace spaces with '+' for the search query
    search_query = search_query.replace(' ', '+')
    url = f'https://www.amazon.com/s?k={search_query}'

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36',
        'Accept-Language': 'en-US, en;q=0.9',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8',
        'Referer': 'https://www.amazon.com/',
    }

    # Send a GET request to fetch the page content
    response = requests.get(url, headers=headers)

    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    products = []
    count = 0

    # Loop through search results and extract product data
    for product in soup.find_all('div', {'data-component-type': 's-search-result'}):
        # Extract product name
        name = product.h2.text.strip()

        # Extract image URL
        image_tag = product.find('img', {'class': 's-image'})
        image_url = image_tag['src'] if image_tag else None

        if name and image_url and count < 5:
            # Save image locally
            image_response = requests.get(image_url)
            image_path = f'product_images/halloween/{name[:30].replace(" ", "_")}.jpg'
            with open(image_path, 'wb') as f:
                f.write(image_response.content)

            products.append((name, image_path))
            count += 1

    return products

# Searching for "halloween"
products = scrape_products("halloween")

# Display product names and image paths
print("Scraped Products:")
for product_name, image_path in products:
    print(f'Product: {product_name}, Image saved at: {image_path}')


Scraped Products:
Product: Halloween Bulk Assorted Fruit Candy - Starburst, Skittles, Gummy Life Savers, Air Heads, Jolly Rancher, Sour Punch, Haribo Gold-Bears, Gummy Bears & Twizzlers (32 Oz Variety Pack), Image saved at: product_images/halloween/Halloween_Bulk_Assorted_Fruit_.jpg
Product: Floating Candles with Wand, Halloween Decoration 20 PCs Magic Hanging Candles, Flickering Warm Light Flameless Floating LED Candle with Wand Remote, Battery Operated Taper Candle Set for Theme Party, Image saved at: product_images/halloween/Floating_Candles_with_Wand,_Ha.jpg
Product: Ring Pop Halloween Bulk Variety Candy - 50 Ct Individually Wrapped Lollipops w/ Assorted Flavors - Fun Candy For Party Favors, Halloween Parties, Trick or Treat Goodie Bags, Bachelorette Parties, Image saved at: product_images/halloween/Ring_Pop_Halloween_Bulk_Variet.jpg
Product: JOYIN Halloween Outdoor Decorations Hairy Spider, Scary Giant Spider Fake Large Props for Yard Party Decor, Black (Standing Height is 15 inch

In [44]:
# Mohamed Yasin

# import Libraries
import requests
from bs4 import BeautifulSoup
import os

# Create a directory to save images
os.makedirs('product_images/toys', exist_ok=True)

# Function to scrape products
def scrape_products(search_query):
    # Replace spaces with '+' for the search query
    search_query = search_query.replace(' ', '+')
    url = f'https://www.amazon.com/s?k={search_query}'

    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36',
        'Accept-Language': 'en-US, en;q=0.9',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8',
        'Referer': 'https://www.amazon.com/',
    }

    # Send a GET request to fetch the page content
    response = requests.get(url, headers=headers)

    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    products = []
    count = 0

    # Loop through search results and extract product data
    for product in soup.find_all('div', {'data-component-type': 's-search-result'}):
        # Extract product name
        name = product.h2.text.strip()

        # Extract image URL
        image_tag = product.find('img', {'class': 's-image'})
        image_url = image_tag['src'] if image_tag else None

        if name and image_url and count < 5:
            # Save image locally
            image_response = requests.get(image_url)
            image_path = f'product_images/toys/{name[:30].replace(" ", "_")}.jpg'
            with open(image_path, 'wb') as f:
                f.write(image_response.content)

            products.append((name, image_path))
            count += 1

    return products

# Searching for "toys"
products = scrape_products("toys")

# Display product names and image paths
print("Scraped Products:")
for product_name, image_path in products:
    print(f'Product: {product_name}, Image saved at: {image_path}')

Scraped Products:
Product: JOYIN 3 Pack Monster Truck Toy - Motion Activated Light-Up Cars for Toddlers - Monster Treads Lightning Wheels - Baby Toy Press & Go Cars for Boys Girls, Image saved at: product_images/toys/JOYIN_3_Pack_Monster_Truck_Toy.jpg
Product: LCD Writing Tablet for Kids, Colorful Toddlers Toys Drawing Board, Educational Kid Toys, Doodle Pad Dinosaur Toys for 2 3 4 5 6 7 8 Year Old Boys Girls Birthday Party Christmas Gifts,8.5inch, Image saved at: product_images/toys/LCD_Writing_Tablet_for_Kids,_C.jpg
Product: PicassoTiles Magnetic Tiles 60pcs Kids Toys Classroom Sensory Toy for Toddlers STEM Learning Building Blocks, Montessori Pretend Play Magnet Tile Construction Stacking Block Boys Girls Ages 3+ PT60, Image saved at: product_images/toys/PicassoTiles_Magnetic_Tiles_60.jpg
Product: Gionlion Friendship Bracelet Kit Concert Bracelets Merch, 5200 Pcs Clay Beads Bracelet Making Kit for Beginner, DIY Arts and Crafts Teen Girl Gifts Toys for Ages 6-13, Image saved at: prod

In [45]:
# Pierrette U

import requests
from bs4 import BeautifulSoup
import os
import time
import random

# Create a directory to save product images
os.makedirs('product_images/decor', exist_ok=True)

# Function to scrape products from Amazon search results
def scrape_amazon_products(search_query):
    """
    Scrapes Amazon search results for a given search query.

    Arguments:
        search_query (str): The product search term to query on Amazon.

    Returns:
        list: A list of product titles and image file paths.
    """
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.102 Safari/537.36',
        'Accept-Language': 'en-US,en;q=0.9',
        'Accept-Encoding': 'gzip, deflate, br',
        'Referer': 'https://www.amazon.com/',
        'Connection': 'keep-alive'
    }

    # Replace spaces with '+' for the search URL
    url = f"https://www.amazon.com/s?k={search_query.replace(' ', '+')}"

    # Send a GET request with headers to avoid blocking
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        # Parse the response content with BeautifulSoup
        soup = BeautifulSoup(response.content, 'html.parser')

        # Find all product containers
        products = soup.find_all('div', {'data-component-type': 's-search-result'})

        product_details = []  # List to store product titles and image paths

        # Iterate over the products
        for product in products[:5]:  # Limit to 5 products
            try:
                # Extract product title
                title = product.h2.text.strip()

                # Extract image URL
                image_tag = product.find('img', {'class': 's-image'})
                image_url = image_tag['src']

                # Save the image locally
                img_data = requests.get(image_url).content
                img_filename = f"product_images/decor/{title[:20].replace(' ', '_')}.jpg"  # Shorten title for filename
                with open(img_filename, 'wb') as handler:
                    handler.write(img_data)

                # Append product details
                product_details.append({
                    'title': title,
                    'image': img_filename
                })

                # Add a random delay between requests to avoid getting blocked
                time.sleep(random.uniform(1, 5))

            except Exception as e:
                print(f"Error processing product: {e}")
                continue

        return product_details
    elif response.status_code == 503:
        print(f"Service unavailable (503), trying again after a short delay...")
        time.sleep(10)  # Wait and retry after 10 seconds
        return scrape_amazon_products(search_query)  # Recursive retry

    else:
        print(f"Failed to retrieve Amazon page, status code: {response.status_code}")
        return []

# Example: Scrape for "decor"
products = scrape_amazon_products('Decor')

# Print product details
for product in products:
    print(f"Product: {product['title']}, Image saved at: {product['image']}")


Product: Sullivans White Ceramic Vase Set, Farmhouse Decor, Home Decorative Vase, Vases For Your Kitchen, Bedroom, Office, Living Room, Bathroom, & Shelf Centerpiece Table Decorations (CM2333), Image saved at: product_images/decor/Sullivans_White_Cera.jpg
Product: Rossetta Star Projector, Galaxy Projector for Bedroom, LED Night Light Aurora Projector with Bluetooth Speaker, White Noise, Timer and Remote, Room Decor, Gifts for Kids, Adults, Christmas, Birthday, Image saved at: product_images/decor/Rossetta_Star_Projec.jpg
Product: Crystal Tree of Life 7 Chakra Healing Crystal Trees for Home Decor, Office Desk Decor, Living Room Decor, Handmade Bonsai Trees for Positive Energy, Money, Good Luck Birthday Gifts for Women, Mom, Image saved at: product_images/decor/Crystal_Tree_of_Life.jpg
Product: LEGO Icons Dried Flower Centerpiece, Botanical Collection Crafts Set for Adults, Artificial Flowers with Rose and Gerbera, Table or Wall Decoration, Home Décor, 10314, Image saved at: product_imag

In [57]:
# Aime

import requests
from bs4 import BeautifulSoup
import os
import time
import random

# Create a directory to save product images
os.makedirs('product_images/watches', exist_ok=True)

# Function to scrape products from Amazon search results
def scrape_amazon_products(search_query):
    """
    Scrapes Amazon search results for a given search query.

    Arguments:
        search_query (str): The product search term to query on Amazon.

    Returns:
        list: A list of product titles and image file paths.
    """
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.102 Safari/537.36',
        'Accept-Language': 'en-US,en;q=0.9',
        'Accept-Encoding': 'gzip, deflate, br',
        'Referer': 'https://www.amazon.com/',
        'Connection': 'keep-alive'
    }

    # Replace spaces with '+' for the search URL
    url = f"https://www.amazon.com/s?k={search_query.replace(' ', '+')}"

    # Send a GET request with headers to avoid blocking
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        # Parse the response content with BeautifulSoup
        soup = BeautifulSoup(response.content, 'html.parser')

        # Find all product containers
        products = soup.find_all('div', {'data-component-type': 's-search-result'})

        product_details = []  # List to store product titles and image paths

        # Iterate over the products
        for product in products[:5]:  # Limit to 5 products
            try:
                # Extract product title
                title = product.h2.text.strip()

                # Extract image URL
                image_tag = product.find('img', {'class': 's-image'})
                image_url = image_tag['src']

                # Save the image locally
                img_data = requests.get(image_url).content
                img_filename = f"product_images/watches/{title[:20].replace(' ', '_')}.jpg"  # Shorten title for filename
                with open(img_filename, 'wb') as handler:
                    handler.write(img_data)

                # Append product details
                product_details.append({
                    'title': title,
                    'image': img_filename
                })

                # Add a random delay between requests to avoid getting blocked
                time.sleep(random.uniform(1, 5))

            except Exception as e:
                print(f"Error processing product: {e}")
                continue

        return product_details
    elif response.status_code == 503:
        print(f"Service unavailable (503), trying again after a short delay...")
        time.sleep(10)  # Wait and retry after 10 seconds
        return scrape_amazon_products(search_query)  # Recursive retry

    else:
        print(f"Failed to retrieve Amazon page, status code: {response.status_code}")
        return []

# Example: Scrape for "watches"
products = scrape_amazon_products('watches')

# Print product details
for product in products:
    print(f"Product: {product['title']}, Image saved at: {product['image']}")


Product: Casio, Image saved at: product_images/watches/Casio.jpg
Product: Apple, Image saved at: product_images/watches/Apple.jpg
Product: SAMSUNG, Image saved at: product_images/watches/SAMSUNG.jpg
Product: Casio, Image saved at: product_images/watches/Casio.jpg
Product: Casio, Image saved at: product_images/watches/Casio.jpg


Zip Files

In [58]:
!zip -r product_images.zip product_images

  adding: product_images/ (stored 0%)
  adding: product_images/laptops/ (stored 0%)
  adding: product_images/laptops/HP_Newest_255_G10_Laptop_for_H.jpg (deflated 1%)
  adding: product_images/laptops/HP_Portable_Laptop,_Student_an.jpg (deflated 3%)
  adding: product_images/laptops/Acer_Aspire_3_A315-24P-R7VH_Sl.jpg (deflated 3%)
  adding: product_images/laptops/Acer_Aspire_Go_15_Slim_Laptop_.jpg (deflated 3%)
  adding: product_images/laptops/HP_Newest_14"_Ultral_Light_Lap.jpg (deflated 3%)
  adding: product_images/halloween/ (stored 0%)
  adding: product_images/halloween/JOYIN_Halloween_Outdoor_Decora.jpg (stored 0%)
  adding: product_images/halloween/Halloween_Bulk_Assorted_Fruit_.jpg (deflated 0%)
  adding: product_images/halloween/Ring_Pop_Halloween_Bulk_Variet.jpg (deflated 0%)
  adding: product_images/halloween/JOYIN_144_Pcs_Glow_Sticks_Bulk.jpg (deflated 0%)
  adding: product_images/halloween/Floating_Candles_with_Wand,_Ha.jpg (stored 0%)
  adding: product_images/watches/ (stored 

In [59]:
from google.colab import files
files.download("product_images.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>